In [11]:
from django.db import connection

from cinema.models import Award, Person, Movie, MovieNomination, PersonNomination
from cinema.choices import MOVIE_GENRES
from cinema.utils import print_sql
from cinema.populate_database import populate_database

In [2]:
from django.db import reset_queries


populate_database()

# Reset queries so they won't impact future output
reset_queries()

# Começando pelo básico

## Filter

https://books.agiliq.com/projects/django-orm-cookbook/en/latest/join.html

In [17]:
queryset = Movie.objects.filter(genre=MOVIE_GENRES.action)

print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id",
       "cinema_movie"."budget"
FROM "cinema_movie"
WHERE "cinema_movie"."genre" = action



## Exclude

In [18]:
queryset = Movie.objects.exclude(genre__in=[MOVIE_GENRES.action, MOVIE_GENRES.mistery])

print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id",
       "cinema_movie"."budget"
FROM "cinema_movie"
WHERE NOT ("cinema_movie"."genre" IN (action,
                                      mistery))



## Filtrando "através" de tabelas
https://docs.djangoproject.com/en/4.0/topics/db/queries/#lookups-that-span-relationships

In [14]:
queryset = Movie.objects.filter(director__name="Greta Gerwig")
# str(queryset.query)
print_sql(queryset)

SELECT "cinema_movie"."title",
       "cinema_person"."name"
FROM "cinema_movie"
INNER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")
WHERE "cinema_person"."name" = Greta Gerwig



## Cuidado com alguns detalhes

https://stackoverflow.com/questions/8164675/chaining-multiple-filter-in-django-is-this-a-bug/28253623#28253623

Doc do Django: https://docs.djangoproject.com/en/3.2/topics/db/queries/#spanning-multi-valued-relationships

In [26]:
queryset_1 = Person.objects.filter(
    movies_directed__title__icontains="batman", movies_directed__release_date__year__gt=2020
)

print_sql(queryset_1)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date"
FROM "cinema_person"
INNER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
WHERE ("cinema_movie"."release_date" > 2020-12-31
       AND UPPER("cinema_movie"."title"::text) LIKE UPPER(%batman%))



In [27]:
queryset_2 = Person.objects.filter(movies_directed__title__icontains="batman").filter(
    movies_directed__release_date__year__gt=2020
)
print_sql(queryset_2)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date"
FROM "cinema_person"
INNER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
INNER JOIN "cinema_movie" T3 ON ("cinema_person"."id" = T3."director_id")
WHERE (UPPER("cinema_movie"."title"::text) LIKE UPPER(%batman%)
       AND T3."release_date" > 2020-12-31)



queryset_1 vai filtrar por diretores que já dirigiram filmes com "batman" no título **e** lançados depois de 2020.

Já queryset_2 vai filtrar por diretores que tanto já dirigiram filmes com "batman" no título quanto filmes lançados depois de 2020.

Isso acontece por causa da forma como o Django processa o filter; tudo dentro de uma única chamada filter() é aplicado simultaneamente para filtrar os itens que correspondem a todos esses requisitos. Já chamadas filter() sucessivas restringem ainda mais o conjunto de objetos, mas para relações multivaloradas, elas se aplicam a qualquer objeto vinculado ao modelo primário, não necessariamente àqueles objetos que foram selecionados por uma chamada filter() anterior.

**Obs**: O mesmo não acontece com .exclude(). Tudo dentro de uma única chamada exclude() não se refere necessariamente ao mesmo item. Para conseguir dar um exclude em objetos que vão dar match com as condições dadas, é preciso realizar duas consultas.

Exemplo da doc (adaptar):
Blog.objects.exclude(
    entry__in=Entry.objects.filter(
        headline__contains='Lennon',
        pub_date__year=2008,
    ),
)

# Problema de N+1

O ORM do Django faz com que a gente interaja com o banco de dados relacional de forma que parece natural pro paradigma de orientação a objetos. Então se no nosso código a gente faz algo como isso aqui, onde temos o queryset de um model e depois iteramos pelo queryset acessando um atributo do objeto e um atributo de um objeto referenciado pelo objeto, isso parece ser certo.

In [4]:
reset_queries()

queryset = Movie.objects.all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

Deep Water was directed by Adrian Lyne


Mas, na verdade, para cada iteração do for, a gente tá acessando o banco de dados duas vezes (fazendo duas queries). A gente pode conferir isso usando o `connection.queries`:

In [5]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" WHERE "cinema_person"."id" = 36 LIMIT 21',
  'time': '0.000'}]

## select_related

O Django sabe do problema e ele dá uma solução para isso, que é o `select_related`. E é usado da seguinte forma: se você vai iterar por um queryset de um model e você precisa acessar um atributo de um campo desse model que é uma referência a uma outra tabela (um Foreign Key field), você chama o select_related depois do objects e passa o nome do campo.

In [6]:
reset_queries()

queryset = Movie.objects.select_related("director").all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

Deep Water was directed by Adrian Lyne


O que está sendo feito ao usar o select_related é que, no SQL usado, o SELECT vai conter os campos do model do queryset e os campos daquele campo que você vai acessar. Como pode ser visto aqui:

In [7]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.001'}]

In [5]:
reset_queries()

queryset = Movie.objects.select_related("director").only("title", "director").all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

connection.queries

Deep Water was directed by Adrian Lyne


[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."director_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.001'}]

Se você precisar "limpar" a lista dos campos relacionados que foram adicionados por chamadas do select_related, basta usar `select_related(None)`.

## prefetch_related

Works for FK relationships, but also for M2M relation ships. However, the optimization is made by Python and not the SQL.

In [6]:
reset_queries()

queryset = Movie.objects.all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" = 3',
  'time': '0.001'}]

In [7]:
reset_queries()

queryset = Movie.objects.prefetch_related("cast").all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT ("cinema_movie_cast"."movie_id") AS "_prefetch_related_val_movie_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" IN (3)',
  'time': '0.001'}]

Saber usar select_related e prefetch_related é algo muito importante para as pessoas que usam Django, porque é fácil cair no problema de N+1 ou problemas similares e ao olhar pro código você pode não sabe que ali tem um problema de performance.

## .count() e .exists()

São duas coisas diferentes que as pessoas às vezes "confundem" e que podem acabar prejudicando a performance da sua aplicação se você não se ligar.

Acontece às vezes da gente usar `queryset.count() > 0` em vez de `queryset.exists()` e não perceber que isso pode ter um impacto negativo.
Acontece que o count vai contar todas as linhas daquela tabela, enquanto o exists apenas checa se existe pelo menos uma linha naquela tabela. Se você tá lidando com um queryset que tem muitos objetos, o count se torna um problema.

`.count()` a gente deve usar apenas se a gente quiser saber o número de objetos num queryset.
`.exists()` a gente deve usar para saber se aquele queryset não é vazio.

(Fazer comparação de tempo entre um e outro)

In [16]:
Movie.objects.all().count()

connection.queries

[{'sql': 'SELECT COUNT(*) AS "__count" FROM "cinema_movie"', 'time': '0.005'}]

In [17]:
reset_queries()

Movie.objects.filter(title="test").exists()

connection.queries

[{'sql': 'SELECT (1) AS "a" FROM "cinema_movie" WHERE "cinema_movie"."title" = \'test\' LIMIT 1',
  'time': '0.000'}]

## Retornar apenas o necessário!

`only`, `defer`, `values` e `values_list` vão te ajudar com isso.

In [3]:
queryset = Movie.objects.all()

print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id",
       "cinema_movie"."budget"
FROM "cinema_movie"



In [4]:
print_sql(queryset.only("title"))

SELECT "cinema_movie"."id",
       "cinema_movie"."title"
FROM "cinema_movie"



In [5]:
print_sql(queryset.defer("title"))

SELECT "cinema_movie"."id",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id",
       "cinema_movie"."budget"
FROM "cinema_movie"



In [6]:
print_sql(queryset.values_list("title"))
queryset.values_list("title")

SELECT "cinema_movie"."title"
FROM "cinema_movie"



<QuerySet [('Deep Water',)]>

In [7]:
print_sql(queryset.values("title"))
queryset.values("title")

SELECT "cinema_movie"."title"
FROM "cinema_movie"



<QuerySet [{'title': 'Deep Water'}]>

## Annotate

Dá mais poder ao desenvolvedor.

### Count

Exemplo: você quer uma lista ordenada a partir de uma contagem específica.

In [8]:
from django.db.models import Count

queryset = Person.objects.annotate(total_movies_directed=Count("movies_directed")).order_by("total_movies_directed")

print_sql(queryset)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date",
       COUNT("cinema_movie"."id") AS "total_movies_directed"
FROM "cinema_person"
LEFT OUTER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
GROUP BY "cinema_person"."id"
ORDER BY "total_movies_directed" ASC



### Count + filter

In [21]:
from django.db.models import Q

queryset = Person.objects.annotate(
    total_movies_directed=Count("movies_directed", filter=Q(movies_directed__genre=MOVIE_GENRES.drama))
)

print_sql(queryset)



SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date",
       COUNT("cinema_movie"."id") FILTER (
                                          WHERE "cinema_movie"."genre" = drama) AS "total_movies_directed"
FROM "cinema_person"
LEFT OUTER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
GROUP BY "cinema_person"."id"



### Subquery e OuterRef

In [16]:
from django.db.models import OuterRef, Subquery, Q

q = Person.objects.annotate(
    latest_starred_movie_title=Subquery(
        Movie.objects.filter(cast=OuterRef("id")).order_by("-release_date").values("title")[:1]
    )
).values_list("name", "latest_starred_movie_title")


# q = Award.objects.annotate(
#     most_nominated_movie=Subquery(
#         Movie.objects.annotate(
#             nominations_count=Count('nominations', filter=Q(nominations__award_id=OuterRef('id')))
#         ).order_by('-nominations_count')[:1]
#     )
# )

print_sql(q)

SELECT "cinema_person"."name",

  (SELECT U0."title"
   FROM "cinema_movie" U0
   INNER JOIN "cinema_movie_cast" U1 ON (U0."id" = U1."movie_id")
   WHERE U1."person_id" = "cinema_person"."id"
   ORDER BY U0."release_date" DESC
   LIMIT 1) AS "latest_starred_movie_title"
FROM "cinema_person"

